In [1]:
import sys, os
from tqdm import tqdm
sys.path.append(R"D:\GITHUB\FunAudioLLM-APP\cosyvoice")
sys.path.append(R"D:\GITHUB\FunAudioLLM-APP\cosyvoice\third_party\Matcha-TTS")

from cosyvoice.cli.cosyvoice import CosyVoice, CosyVoice2
from cosyvoice.utils.file_utils import load_wav
from cosyvoice.utils.common import set_all_random_seed
import torchaudio

2025-02-24 00:57:08,473 - modelscope - INFO - PyTorch version 2.3.1+cu121 Found.
2025-02-24 00:57:08,476 - modelscope - INFO - Loading ast index from C:\Users\CHENHAOYUAN\.cache\modelscope\ast_indexer
2025-02-24 00:57:08,584 - modelscope - INFO - Loading done! Current index file version is 1.15.0, with md5 553a05bea23a95025342c8774cbd7108 and a total number of 980 components indexed


failed to import ttsfrd, use WeTextProcessing instead


## 加载模型

In [2]:
cosyvoice = CosyVoice2(R'D:\GITHUB\FunAudioLLM-APP\cosyvoice\pretrained_models\CosyVoice2-0.5B', load_jit=False, load_trt=False, fp16=False)

d:\anaconda3\envs\cosyvoice\lib\site-packages\diffusers\models\lora.py:393: FutureWarning: `LoRACompatibleLinear` is deprecated and will be removed in version 1.0.0. Use of `LoRACompatibleLinear` is deprecated. Please switch to PEFT backend by installing PEFT: `pip install peft`.
  deprecate("LoRACompatibleLinear", "1.0.0", deprecation_message)
2025-02-24 00:57:19,106 INFO input frame rate=25
d:\anaconda3\envs\cosyvoice\lib\site-packages\onnxruntime\capi\onnxruntime_inference_collection.py:69: UserWarning: Specified provider 'CUDAExecutionProvider' is not in available provider names.Available providers: 'AzureExecutionProvider, CPUExecutionProvider'
  warnings.warn(
2025-02-24 00:57:20,735 WETEXT INFO building fst for zh_normalizer ...
2025-02-24 00:57:20,735 INFO building fst for zh_normalizer ...
2025-02-24 00:57:43,918 WETEXT INFO done
2025-02-24 00:57:43,918 INFO done
2025-02-24 00:57:43,919 WETEXT INFO fst path: d:\anaconda3\envs\cosyvoice\lib\site-packages\tn\zh_tn_tagger.fst
202

## 加载音频
需要手动标记一下音频的内容

In [3]:
prompt_audio = load_wav(R'D:\GITHUB\OpenVoice\inputs\train\tvboy1_denoised.wav', 16000)
prompt_text = "两个或者三个吧，嗯肚子很胀，就是，有点，嗯，有时候会吐的"

## 配置

In [4]:
batch = 4 # doesn't work at this moment
output_dir = "./assets/raw_dataset/tvboy_denoised_mandarin_v2_valid_seed34751218"
seed = 34751218

# 以下选一个

## 普通话
input_txts = "./assets/text/mandarin_v2_valid.txt"
cross_lang = ""
skip_line = 1 # 跟数据集版本有关，v1为0，v2为1

## 粤语
# input_txts = "./cantonese.txt"
# cross_lang = "粤语"

# ！！下面的不要动

if not os.path.exists(output_dir):
	os.makedirs(output_dir, exist_ok=True)
with open(input_txts, 'r', encoding='utf-8') as f:
	lines = f.readlines()
	input_text_lines = []
	for i in range(0, len(lines), skip_line+1):
		line = lines[i].strip()
		if line == "": continue
		input_text_lines.append(line)
	print(input_text_lines[:5])
	print("Got dataset: ", len(input_text_lines))
	def input_generator():
		for i in range(0, len(input_text_lines), batch):
			yield input_text_lines[i:min(i+batch, len(input_text_lines))]
set_all_random_seed(seed)
print("Output folder: ", output_dir)
print("Using seed: ", seed)

['今年在外包装上还首次有了防伪记号。', '既想让马儿跑，又不给马儿足够的草，其结果也就在意料之中。', '最后，张金中在漆黑的夜色下涮洗。', '赵云的朋友郝超英建议先带董静去海口躲躲。', '救援队正在加快安装固定泵，通过抽水稳定水位。']
Got dataset:  30
Output folder:  ./assets/raw_dataset/tvboy_denoised_mandarin_v2_valid_seed34751218
Using seed:  34751218


## 测试推理

In [7]:
sample_text = "从A到B需要C来维持，然而若D到E然后从F到G就会让H达不到I隔壁的J。" #K、L、M、N、O、P、Q、R、S、T、U、V、W、X、Y、Z。" # input_text_lines[0]
def zero_infer():
	print("Using zero infer.")
	ret = list(cosyvoice.inference_zero_shot(sample_text, prompt_text, prompt_audio, stream=False))[0]
	torchaudio.save("zero_infer.wav", ret['tts_speech'], cosyvoice.sample_rate)
def cross_infer():
	print("Using cross shot to " + cross_lang + ".")
	instruct_text = f"用{cross_lang}说这句话"
	ret = list(cosyvoice.inference_instruct2(sample_text, instruct_text, prompt_audio, stream=False))[0]
	torchaudio.save("cross_infer.wav", ret['tts_speech'], cosyvoice.sample_rate)
if cross_lang == "":
	zero_infer()
else:
	cross_infer()
print("Done.")

Using zero infer.


  0%|          | 0/1 [00:00<?, ?it/s]

min value is  tensor(-1.0223)
max value is  tensor(1.0314)


2025-02-24 00:38:34,547 INFO synthesis text 从A到B需要C来维持，然而若D到E然后从F到G就会让H达不到I隔壁的J。
d:\anaconda3\envs\cosyvoice\lib\site-packages\transformers\models\qwen2\modeling_qwen2.py:544: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:455.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(
We detected that you are passing `past_key_values` as a tuple of tuples. This is deprecated and will be removed in v4.47. Please convert your cache or use an appropriate `Cache` class (https://huggingface.co/docs/transformers/kv_cache#legacy-cache-format)
2025-02-24 00:38:44,181 INFO yield speech len 13.8, rtf 0.6980435399041659
100%|██████████| 1/1 [00:10<00:00, 10.38s/it]

Done.


## 生成

In [5]:
SkipExisted = True
def zero_shot():
	print("Using zero shot.")
	for idx, input_data in enumerate(tqdm(input_generator())):
		for subidx, data in enumerate(input_data):
			global_idx = idx * batch + subidx + 1
			filename = os.path.join(output_dir, f"{global_idx}.wav")
			if SkipExisted and os.path.exists(filename):
				print("Skip ", filename)
				continue
			ret = list(cosyvoice.inference_zero_shot(data, prompt_text, prompt_audio, stream=False))[0]
			torchaudio.save(filename, ret['tts_speech'], cosyvoice.sample_rate)
def cross_shot():
	print("Using cross shot to " + cross_lang + ".")
	instruct_text = f"用{cross_lang}说这句话"
	for idx, input_data in enumerate(tqdm(input_generator())):
		for subidx, data in enumerate(input_data):
			global_idx = idx * batch + subidx + 1
			filename = os.path.join(output_dir, f"{global_idx}.wav")
			if SkipExisted and os.path.exists(filename):
				print("Skip ", filename)
				continue
			ret = list(cosyvoice.inference_instruct2(data, instruct_text, prompt_audio, stream=False))[0]
			torchaudio.save(filename, ret['tts_speech'], cosyvoice.sample_rate)
if cross_lang == "":
	zero_shot()
else:
	cross_shot()
print("Done.")

Using zero shot.


0it [00:00, ?it/s]

min value is  tensor(-1.0223)
max value is  tensor(1.0314)


2025-02-24 00:57:47,648 INFO synthesis text 今年在外包装上还首次有了防伪记号。
d:\anaconda3\envs\cosyvoice\lib\site-packages\transformers\models\qwen2\modeling_qwen2.py:544: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:455.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(
We detected that you are passing `past_key_values` as a tuple of tuples. This is deprecated and will be removed in v4.47. Please convert your cache or use an appropriate `Cache` class (https://huggingface.co/docs/transformers/kv_cache#legacy-cache-format)
2025-02-24 00:57:51,937 INFO yield speech len 5.16, rtf 0.8312660594319188
100%|██████████| 1/1 [00:04<00:00,  4.96s/it]


min value is  tensor(-1.0223)
max value is  tensor(1.0314)


...

min value is  tensor(-1.0223)
max value is  tensor(1.0314)


2025-02-24 00:59:53,571 INFO synthesis text 他能率领队友守住澳大利亚队的疯狂进攻吗？
2025-02-24 00:59:57,906 INFO yield speech len 6.52, rtf 0.6646285027814058
100%|██████████| 1/1 [00:04<00:00,  4.91s/it]


min value is  tensor(-1.0223)
max value is  tensor(1.0314)


2025-02-24 00:59:58,495 INFO synthesis text 王奇孤身一人，无儿无女，靠政府补助和拾荒度日。
2025-02-24 01:00:02,381 INFO yield speech len 5.48, rtf 0.7089962489413518
100%|██████████| 1/1 [00:04<00:00,  4.46s/it]
8it [02:15, 16.93s/it]

Done.
